## Libraries

In [97]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import en
#import itertools

## Read input file

In [98]:
df = pd.read_csv('sams.csv')

## Train algorithm

In [99]:
#Training the algorithm
#To-do: Instead of speech train it on actual test case corpus
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
train_text = state_union.raw("2005-GWBush.txt")
custom_sent_tokenizer = PunktSentenceTokenizer(train_text)

## fn: Rule engine for multiple 'should' statement

In [100]:
def transform_stmt_mul_shd(stmt):
    temp_pst_collate = ""
    splt_sentence = stmt.split("and")
    for stmt in splt_sentence:
        temp_pst_collate = temp_pst_collate + 'and' + transform_stmt_single_shd(stmt)
    return temp_pst_collate[4:]          

## fn: Rule engine for single 'should' statement

In [101]:
def transform_stmt_single_shd(stmt): 
    words = nltk.word_tokenize(stmt)
    veb_lst = [word for (word, pos) in nltk.pos_tag(words) if is_verb(pos)]
    
    if (stmt.find("should not") > -1) and ('be' in veb_lst):
        trnf_pst_stmt = stmt.replace("should not be", "were not")
        return trnf_pst_stmt
    
    elif (stmt.find("should") > -1) and ('be' in veb_lst) and (stmt.find("fields") > -1):
        trnf_pst_stmt = stmt.replace("should be", "were")
        return trnf_pst_stmt
    
    elif (stmt.find("should") > -1) and ('be' in veb_lst):
        trnf_pst_stmt = stmt.replace("should be", "was")
        return trnf_pst_stmt
    
    elif (stmt.find("should") > -1) and ('be' not in veb_lst):
        trnf_pst_stmt = stmt.replace("should "+veb_lst[0], en.verb.past(veb_lst[0].lower()))
        return trnf_pst_stmt
    
    else:
        return stmt

In [102]:
# creating a function that will retrive only base verb forms
is_verb = lambda pos: pos == 'VB' 

In [103]:
#verb_lst = []
all_past_stmt = []
temp_pst = ""

for test_case in list(df["Expect_res"]):
    stmt_lst = custom_sent_tokenizer.tokenize(test_case)
    try:
        for stmt in stmt_lst:
            words = nltk.word_tokenize(stmt)
            if words.count("should") > 1:
                temp_pst = temp_pst + '\n' + transform_stmt_mul_shd(stmt)
            else:
                temp_pst = temp_pst + '\n' + transform_stmt_single_shd(stmt)
                
        all_past_stmt.append(temp_pst[1:])
        temp_pst = ""

    except Exception as e:
        all_past_stmt.append(test_case)
        print(str(e))

## Adding column to df

In [104]:
se = pd.Series(all_past_stmt)

In [105]:
df['out'] = se.values

In [106]:
#returns the dimesnsion of the data
df.shape

(942, 4)

In [107]:
#returns number of lines in the list
len(se)

942

In [108]:
# display dimension of the df after adding out column
df.shape

(942, 4)

In [109]:
#df['out']

## Create new output file

In [110]:
df.to_csv('FinalOut_1.csv', encoding='utf-8')